In [1]:
import folium
import warnings
warnings.filterwarnings("ignore")

In [2]:
loc=[37.5662952, 126.9779451] #위도,경도

In [3]:
map = folium.Map(location=loc, zoom_start=16)
folium.Marker(location=loc).add_to(map)
map

In [4]:
import folium
import requests
from pandas import json_normalize

In [5]:
loc=[37.5662952, 126.9779451] #위도,경도
targetSite='https://www.starbucks.co.kr/store/getStore.do?r=8K3XF06R9Q' #getStore.do 서비스이름
req=requests.post(targetSite, data={
'ins_lat': 37.563398,
'ins_lng': 126.9863309,
'p_sido_cd': '01',    #01=서울특별시
'p_gugun_cd': '',     #빈값이면 전체 검색 
'in_biz_cd': '',      #빈값=스타벅스 매장 전체
'iend': 600,          # 검색반경
'set_date': ''        
})
starbucks=req.json()
print(type(starbucks))


<class 'dict'>


In [ ]:
st_df=json_normalize(starbucks,'list') 
#starbucks 안에는 list라는 키(key)가 있고, 그 안에 실제 매장 데이터 리스트가 들어있습니다.
st_df.head()
#list'를 지정하면, 매장별 정보 하나하나를 행(row)으로 변환합니다.

,seq,sido_cd,sido_nm,gugun_cd,gugun_nm,code_order,view_yn,store_num,sido,gugun,...,t30,t36,t27,t29,t43,t48,z9999,t64,t66,p02
0,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
1,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
2,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
3,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
4,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
st_df_map = st_df[['s_name','sido_code','sido_name','gugun_code','gugun_name',
'doro_address','lat','lot']]  #"lat"은 위도(latitude), "lot"은 경도(longitude)
st_df_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   s_name        600 non-null    object
 1   sido_code     600 non-null    object
 2   sido_name     600 non-null    object
 3   gugun_code    600 non-null    object
 4   gugun_name    600 non-null    object
 5   doro_address  600 non-null    object
 6   lat           600 non-null    object
 7   lot           600 non-null    object
dtypes: object(8)
memory usage: 37.6+ KB


In [8]:
st_df_map['lat'] = st_df_map['lat'].astype(float)
st_df_map['lot'] = st_df_map['lot'].astype(float)
st_df_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   s_name        600 non-null    object 
 1   sido_code     600 non-null    object 
 2   sido_name     600 non-null    object 
 3   gugun_code    600 non-null    object 
 4   gugun_name    600 non-null    object 
 5   doro_address  600 non-null    object 
 6   lat           600 non-null    float64
 7   lot           600 non-null    float64
dtypes: float64(2), object(6)
memory usage: 37.6+ KB


In [9]:
st_df_map.head(5)

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot
0,역삼아레나빌딩,01,서울,0101,강남구,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069
1,논현역사거리,01,서울,0101,강남구,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223
2,신사역성일빌딩,01,서울,0101,강남구,서울특별시 강남구 강남대로 584 (논현동),37.513931,127.020606
3,국기원사거리,01,서울,0101,강남구,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495
4,대치재경빌딩,01,서울,0101,강남구,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583


In [10]:
loc1 = [37.501087, 127.043069]
stMap = folium.Map(location=loc1, zoom_start=18)
pop = folium.Popup('역삼아레나빌딩', max_width=100)
folium.Marker(location=loc1, popup=pop).add_to(stMap)
stMap

In [12]:
import folium

#st_df_map 데이터프레임이 이미 준비되어 있다고 가정합니다.
#지도의 중심점을 데이터의 평균 위도와 경도로 설정합니다.
map_center = [st_df_map['lat'].mean(), st_df_map['lot'].mean()]
my_map = folium.Map(location=map_center, zoom_start=14)

#for문을 사용해 데이터프레임의 각 행을 순회하며 마커를 추가합니다.
for index, row in st_df_map.iterrows():
    # 현재 행의 위도, 경도, 이름을 변수에 저장합니다.
    lat = row['lat']
    lon = row['lot']
    name = row['s_name']

#위에서 저장한 변수를 사용하여 마커를 생성하고 지도에 추가합니다.
    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(name, max_width=200), # 클릭 시 나타나는 팝업
        tooltip=name # 마우스를 올렸을 때 보이는 이름
    ).add_to(my_map)

#완성된 지도를 표시합니다.
my_map

In [13]:

stmap = folium.Map(location=[st_df_map['lat'].mean(), st_df_map['lot'].mean()], zoom_start=14)

for i, row in st_df_map.iterrows():
    folium.Marker(
        location=[row['lat'], row['lot']],  
        popup=folium.Popup('스타벅스', max_width=100)
    ).add_to(stmap)

stmap